In [2]:
# n_nodes = 'number of nodes'
# main_model = 'Model central'
# model_dict = init_nodes_dict(n_nodes, main_model) dictionnary of local models

import copy
def init_model(n_nodes,main_model):
    return {k:copy.deepcopy(model)}

In [1]:
def send_model(main_model, model_dict, number_of_nodes): #send model to nodes
    with torch.no_grad():
        for i in range(number_of_nodes):
            state_dict= copy.deepcopy(main_model.state_dict())
            model_dict[i].load_state_dict(state_dict)
    return model_dict    


In [ ]:
def fedavg(main_model, model_dict, number_of_nodes): 
    state_dict = model_dict[0].state_dict()
    for name, param in model_dict[0].named_parameters():
        for i in range(1, number_of_nodes):
            state_dict[name]=  state_dict[name] + model_dict[i].state_dict()[name]
        state_dict[name] = state_dict[name]/number_of_nodes
    new_model = copy.deepcopy(model)
    new_model.load_state_dict(state_dict)
    return new_model

In [ ]:
def training_plan(main_model,model_dict,data_load, Round=3, number_of_nodes=2):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")   
    for i in range(Round):
        print('Round '+ str(i+1))
        print('Sending main model to nodes in round '+ str(i+1))
        model_dict = send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_nodes)
        for j in model_dict.keys():
            print('training {} for Round {} :'.format(j,i))
            model = model_dict[j].to(device)
            optimizer = optim.SGD(model.parameters(), lr=0.001)
            scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
            train_loader = trans[j]
            valid_loader =  data_load[j][3]
            train_set =  data_load[j][0]
            valid_set =  data_load[j][1]
            model_dict[j] = train_model(model, criterion, optimizer, scheduler,  train_loader, valid_loader, train_set, valid_set)
        print('Fedavg of main model in round '+ str(i+1))
        main_model = fedavg(main_model, model_dict, number_of_nodes)    
        
    return model_dict, main_model

In [ ]:
def create_model_dict(number_of_nodes):
    model_dict = dict()
    
    for i in range(number_of_nodes):
        model_name="node"+str(i)
        model_info= Mymodel().resnet
        model_dict.update({model_name : model_info })
        
        
    return model_dict

In [ ]:
def send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_nodes): 
    with torch.no_grad():
        for i in range(number_of_nodes):
            state_dict= copy.deepcopy(main_model.state_dict())
            model_dict[name_of_models[i]].load_state_dict(state_dict)
    return model_dict